# Accessing OpenAI Like a Developer (Bonus Assignment)

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [4]:
!pip install openai -q

## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

> NOTE: This requires an OpenAI Key, which can be obtained following [this](https://github.com/AI-Maker-Space/AIE4/tree/main/OpenAI%20API%20Key%20Setup) process.

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")
#openai.api_key=os.environ["OPENAI_API_KEY"]
#dog=os.environ["OPENAI_API_KEY"]
#print(dog)

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [6]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [7]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [8]:
response

ChatCompletion(id='chatcmpl-A3dIBaYAB42P6sHJo7e4E2AsLN8Nz', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", role='assistant', function_call=None, tool_calls=None, refusal=None), logprobs=None)], created=1725429387, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=13, total_tokens=44))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [9]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [10]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [11]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I don't give a damn about the shape of the ice! Just give me something cold to eat before I lose my goddamn mind from hunger!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [12]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I love crushed ice! It just feels so luxurious, doesn't it? Like little icy snowflakes in your drink. How about you, what's your preference?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [13]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I tried to follow the stimple instructions for making falbean stew, but it still turned out tasting strange.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [14]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I need to grab my stimple drill and falbean wrench to fix that loose screw in the cabinet.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shop prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [15]:
### Enrico Manes' EXAMPLE FOR THIS HOMEWORK ASSIGNMENT

!pip install openai -q

import os
import getpass
from IPython.display import display, Markdown

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")
from openai import OpenAI

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message} 

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))   

list_of_prompts = [
    user_prompt("When a person uses the word hate they dislike the movie which is a negative sentiment:"),
    user_prompt("Provide whether the user had positive or negative sentiment towards the movie by placing their comment after INPUT and then the sentiment classifier after OUTPUT:"),
    user_prompt("I loved Deadpool 1, 2, and 3!"),
    assistant_prompt ("'INPUT: I loved Deadpool 1,2, and 3! Output: {sentiment: positive}' "),
    user_prompt("when a person uses the word love or loved they really like the movie which is a positive sentiment:"),
    user_prompt("I loved the original Jaws but hated Jaws 2"),
    user_prompt("I really enjoyed Dodgeball")
]

HW1B_Activity1_Response = get_response(openai_client, list_of_prompts)
pretty_print(HW1B_Activity1_Response)

'INPUT: I loved the original Jaws but hated Jaws 2 - OUTPUT: {sentiment: mixed}'
'INPUT: I really enjoyed Dodgeball - OUTPUT: {sentiment: positive}'

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [19]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

It does matter which travel option Billy selects. If he flies and then takes a bus, it will take a total of 5 hours, getting him home at 6PM local time. If he takes the teleporter and then a bus, it will only take a total of 1 hour, getting him home at 2PM local time. So, if Billy wants to get home before 7PM EDT, he should choose the teleporter option.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [ ]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Let's break it down step by step:

1. Flying (3hrs) + Bus (2hrs) option:
- If Billy chooses to fly, it will take him 3 hours to reach his destination. Since it's currently 1PM local time, he will arrive at 4PM local time.
- After that, he will need to take a bus, which will take an additional 2 hours. This means he will arrive at his destination at 6PM local time.

2. Teleporter (0hrs) + Bus (1hr) option:
- If Billy chooses to take the teleporter, it will take him 0 hours to reach his destination. He will arrive instantly.
- After that, he will need to take a bus, which will take 1 hour. This means he will arrive at his destination at 1PM local time + 1 hour = 2PM local time.

Comparing the two options, it is clear that the Teleporter + Bus option is faster. Billy will arrive at his destination at 2PM local time, which is well before 7PM EDT. Therefore, it does matter which travel option Billy selects, and he should choose the Teleporter + Bus option to ensure he gets home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [31]:
system_template = """\
Think step by step.,
Provide 3 different recipes.,
Answer with a human-like voice.,
Follow the structure Recipe 1, Recipe 2, and Recipe 3
"""
user_template = """{input}
Provide methods to make very good bagels,
Place the ingredients at the top,
Use a structure that has the instructions numbered after the ingredients
"""

# system_template = """\
# WRITE YOUR SYSTEM PROMPT HERE
# """


list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template)
]

bagel_response = get_response(openai_client, list_of_prompts)
pretty_print(bagel_response)

Recipe 1:
Ingredients:
- 4 cups bread flour
- 1 tablespoon sugar
- 1 1/2 teaspoons salt
- 1 tablespoon instant yeast
- 1 1/4 cups warm water
- 1 egg (for egg wash)
- Sesame seeds or poppy seeds (optional)

Instructions:
1. In a large mixing bowl, combine the bread flour, sugar, salt, and instant yeast.
2. Gradually add the warm water to the dry ingredients and mix until a dough forms.
3. Knead the dough on a floured surface for about 10 minutes, or until it is smooth and elastic.
4. Divide the dough into 8 equal pieces and shape each piece into a ball. 
5. Flatten each ball slightly and poke a hole in the center with your finger, stretching it to create a bagel shape.
6. Place the shaped bagels on a parchment-lined baking sheet, cover with a clean towel, and let them rise for 30-60 minutes.
7. Preheat the oven to 425°F (220°C).
8. Bring a large pot of water to a boil and carefully place the risen bagels in the water, boiling for 1-2 minutes on each side.
9. Remove the bagels from the water and place them back on the baking sheet.
10. Brush the tops of the bagels with egg wash and sprinkle with sesame seeds or poppy seeds if desired.
11. Bake in the preheated oven for 20-25 minutes, or until the bagels are golden brown.
12. Cool on a wire rack before serving. Enjoy your freshly baked bagels!

Recipe 2:
Ingredients:
- 3 1/2 cups all-purpose flour
- 1 1/2 teaspoons salt
- 1 tablespoon sugar
- 1 tablespoon active dry yeast
- 1 1/4 cups warm water
- 1 egg (for egg wash)
- Everything bagel seasoning (optional)

Instructions:
1. In a large bowl, combine flour, salt, sugar, and yeast.
2. Gradually add warm water to the dry ingredients, stirring until a dough forms.
3. Knead the dough on a floured surface for about 8-10 minutes, until smooth and elastic.
4. Divide the dough into 8 equal pieces and shape each into a ball, then poke a hole in the center and stretch to form a bagel shape.
5. Place the bagels on a baking sheet lined with parchment paper, cover, and let rise for 30-45 minutes.
6. Preheat the oven to 400°F (200°C).
7. In a large pot, bring water to a boil and carefully place the risen bagels in the water, boiling for 1 minute on each side.
8. Remove the bagels from the water and place them back on the baking sheet.
9. Brush the tops of the bagels with egg wash and sprinkle with everything bagel seasoning if desired.
10. Bake in the preheated oven for 20-25 minutes until golden brown.
11. Cool on a wire rack and enjoy your homemade bagels with your favorite toppings!

Recipe 3:
Ingredients:
- 4 cups high-gluten flour
- 2 teaspoons salt
- 1 tablespoon honey
- 1 packet active dry yeast
- 1 1/2 cups warm water
- Coarse salt for topping

Instructions:
1. In a large mixing bowl, combine high-gluten flour, salt, honey, and yeast.
2. Gradually add warm water and mix until a dough forms.
3. Knead the dough on a floured surface for 8-10 minutes until it is smooth and elastic.
4. Divide the dough into 6-8 pieces and shape into balls, then flatten and shape into bagels.
5. Place the shaped bagels on a baking sheet, cover with a kitchen towel, and let rise for 30-45 minutes.
6. Preheat the oven to 425°F (220°C).
7. Boil a large pot of water and carefully add the risen bagels, boiling for 1-2 minutes on each side.
8. Place the boiled bagels back on the baking sheet and sprinkle with coarse salt.
9. Bake in the preheated oven for 20-25 minutes, or until golden brown.
10. Cool on a wire rack before serving. Enjoy your delicious homemade bagels!

In [ ]:
#user_template = """{input}
#MODIFICATIONS HERE 
#"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [32]:
query = "Provide 3 very good bagel recipes"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

**Recipe 1: Classic Plain Bagels**

Ingredients:
- 1 1/2 cups warm water
- 1 tablespoon active dry yeast
- 4 cups bread flour
- 2 teaspoons salt
- 1 tablespoon sugar
- 1 tablespoon honey
- 1 egg, beaten (for egg wash)
- Optional toppings like sesame seeds, poppy seeds, or everything seasoning

Instructions:
1. In a bowl, combine warm water, yeast, and sugar. Let it sit for 5-10 minutes until foamy.
2. In a separate bowl, mix flour and salt. Gradually add the yeast mixture and honey, then knead until a smooth dough forms.
3. Divide the dough into 8-10 equal parts and shape them into balls. Make a hole in the center and stretch it to form a bagel shape.
4. Preheat the oven to 425°F (220°C) and bring a large pot of water to a boil. Boil each bagel for 1-2 minutes on each side.
5. Place the boiled bagels on a baking sheet lined with parchment paper. Brush the tops with egg wash and sprinkle your desired toppings.
6. Bake for 20-25 minutes until golden brown. Let them cool before enjoying!

**Recipe 2: Cinnamon Raisin Bagels**

Ingredients:
- 1 1/2 cups warm water
- 1 tablespoon active dry yeast
- 4 cups bread flour
- 2 teaspoons salt
- 1/3 cup brown sugar
- 1 1/2 teaspoons cinnamon
- 1 cup raisins
- 1 egg, beaten (for egg wash)

Instructions:
1. Follow steps 1 and 2 from the Classic Plain Bagel recipe.
2. Add brown sugar, cinnamon, and raisins to the dough before kneading.
3. Continue with steps 3 to 6 in the Classic Plain Bagel recipe, adjusting the baking time if needed due to added ingredients.

**Recipe 3: Everything Bagels**

Ingredients:
- 1 1/2 cups warm water
- 1 tablespoon active dry yeast
- 4 cups bread flour
- 2 teaspoons salt
- 1 tablespoon sugar
- 1 tablespoon honey
- 2 tablespoons dried minced garlic
- 2 tablespoons dried minced onion
- 1 tablespoon sesame seeds
- 1 tablespoon poppy seeds
- 1 egg, beaten (for egg wash)

Instructions:
1. Follow steps 1 and 2 from the Classic Plain Bagel recipe.
2. Combine garlic, onion, sesame seeds, and poppy seeds into the dough before shaping the bagels.
3. Proceed with steps 3 to 6 from the Classic Plain Bagel recipe, adjusting baking time as necessary for added toppings. Enjoy your everything bagels!

In [33]:
pretty_print(evaluator_response)

{
"clarity" : 9,
"faithfulness" : 9,
"correctness" : 8
}

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> NOTE: You will have to update and rerun the cells in Step 4 in order to observe any changes.

> PROVIDE YOUR ANSWER HERE : I found that I had to be very specific about providing 3 DIFFERENT recipes to the System Prompt but it also really helped to provide the last prompt line of "Follow the structure Recipe 1, Recipe 2, and Recipe 3"